In [1]:
#install.packages('openxl', repos='http://cran.us.r-project.org')
source('source.r')

load.library('openxl')
read.xlsx = read_excel

Packages Loaded:
ncdf4 R.matlab openxlsx RColorBrewer compiler lattice geosphere readxl data.table rworldmap rworldxtra


Loading required package: ncdf4
Loading required package: R.matlab
Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘R.matlab’”Loading required package: openxlsx
Loading required package: RColorBrewer
Loading required package: compiler
Loading required package: lattice
Loading required package: geosphere
Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘geosphere’”Loading required package: readxl
Loading required package: data.table
Loading required package: rworldmap
Loading required package: sp
### Welcome to rworldmap ###
For a short introduction type : 	 vignette('rworldmap')
Loading required package: rworldxtra


Packages Loaded:
openxl


Loading required package: openxl
Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘openxl’”

[[1]]
[1] FALSE

---
# 2. Load and prepare ship data
This section deals with the loading of the met-merge data. Once loaded and the structure is verified, a 2 minute average is taken for all parameters. Once completed, the next section will add the NCEP wind data and MLD to the data frame.The file structure which this script is looking for is as follows where Vars are in any order but with the proper abbreviations.

``Datetime  Var1   Var2  Var3 ...``

Variable Descriptions and Abbreviations
* __DT -- Date Time__
* __TW -- True Wind__
* TI  -- Wind Dir
* __LA -- Latitude__
* __LO -- Longitude__
* __AT -- Atm Temp__
* __BP -- Pressure__
* PA -- PAR
* __TT (TT2*)  -- Water Temp__
* __SA -- Salinity__
* __OS -- Oxygen__
* FL -- Fluorometer
* BT -- Bottom Depth

___NB___: Bold identies required columns while the others are used for additional analyses.

In [2]:
read.ship.data = function(path) {
    ## Load the data file
    data = read.xlsx(path)
    ## Trim data to the following columns:
    keep = c('DT', 'TW', 'TI', 'LA', 'LO', 'AT', 'BP', 'PA', 'TT', 'SA', 'OS', 'OX', 'FL', 'BT')
    data = data[, which(names(data) %in% keep)]

    # Convert excel datetime numbers to actual datetimes.
    #data$DT = conv_excel_time(data$DT, tz='GMT')

    # Calculated from Thomas et al.
    data$TW = data$TW * 0.942  

    ## Convert all columns (except date/time to numeric)
    #for (i in 2:ncol(data)) {
    #    data[,i] = as.numeric(data[,i])
    #}

    ## Check that it loaded properly/correct data type (num)
    print(paste('Do the number of names in keep match the number of columns now?', ncol(data) == length(keep)))
    
    data
}

In [3]:
options(warn = -1)
ship.data = read.ship.data('./Raw Data/MET/met-merge-final.xlsx')
options(warn = 0)

[1] "Do the number of names in keep match the number of columns now? FALSE"


In [10]:
str(ship.data)
difftime(ship.data$DT[1:10], ship.data$DT[1])

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	185872 obs. of  17 variables:
 $ DT  : POSIXct, format: "2017-05-31 00:01:00" "2017-05-31 00:01:00" ...
 $ TW  : num  2.54 2.17 2.83 3.01 3.01 ...
 $ TI  : num  209 214 216 224 226 ...
 $ TT  : num  19.4 19.4 19.4 19.4 19.4 ...
 $ SA  : num  33.2 33.2 33.2 33.2 33.2 ...
 $ OS  : num  5 5 5 5 5 ...
 $ FL  : num  0.079 0.08 0.077 0.076 0.076 0.077 0.076 0.076 0.076 0.074 ...
 $ AT  : num  16.7 16.7 16.7 16.8 16.8 ...
 $ BP  : num  1010 1010 1010 1010 1010 ...
 $ PA  : num  925 930 918 915 913 ...
 $ SA.1: num  33.2 33.2 33.2 33.2 33.2 ...
 $ OX  : num  -99 -99 -99 -99 -99 -99 -99 -99 -99 -99 ...
 $ OS.1: num  5 5 5 5 5 ...
 $ FL.1: num  0.079 0.08 0.077 0.076 0.076 0.077 0.076 0.076 0.076 0.074 ...
 $ BT  : num  -99 -99 -99 -99 -99 -99 -99 -99 -99 -99 ...
 $ LA  : num  32.7 32.7 32.7 32.7 32.7 ...
 $ LO  : num  -117 -117 -117 -117 -117 ...


Time differences in secs
 [1]   0   0   0  60  60  60  60 120 120 120

#### Remove bad wind data
Here we simply remove the rows there wind speed is less than zero (TW < 0). We also report the number of rows affected (should be a relatively small amount).

In [5]:
bad.wind = which(ship.data$TW < 0)
length(bad.wind)

## Remove bad wind entries
ship.data = ship.data[-bad.wind,]

[1] 618

### The N minute average:
1. Start with the first row, set ___current.time___ to that row's time
2. Find all rows with times >= to the current time
3. Remove all rows with times more than N minutes away from the ___current.time___.
4. Take the column average of those rows and replace the current row.
5. Remove all the other rows
6. Repeat process on the next row.

In [8]:
take.avg = function(data, n = 120) {
    ### N Minute Averaging section

    before = nrow(data)
    i = 1
    nc = ncol(data)

    while (i < nrow(data)) {  # Loop through each row in ship.data
        current.time = data$DT[i]
        
        ## Determine which rows are within N minutes of the current row
        in.range = which(data$DT[1:(i+100)] >= data$DT[i] &
                         difftime(data$DT[1:(i+100)], data$DT[i], units='secs') < n)

        ##  Average the column values together ignoring the first one (time)
        data[i, 2:nc] = apply(data[in.range, 2:nc], 2, function(x) {mean(x, na.rm = TRUE)})

        ## Remove all rows used to make average except for row i
        in.range = in.range[in.range != i]
        if (length(in.range) > 0) {
            data = data[-in.range,]
        }
        i = i + 1
    }

    print(paste('The number of rows before was', before, 'and now there are', nrow(data)))
    data
}

__NB This will take a while.__

In [ ]:
ship.data = take.avg(ship.data, 60)

## Save Results

In [17]:
## Save point after loading ship data and averaging. 
save(ship.data, file='./RStates/ship.avg.rdata')
write.xlsx(ship.data, file='./Input Data/Shipdata - Averaged.xlsx')

gc() # Free up memory

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,760758,40.7,1442291,77.1,1442291,77.1
Vcells,3810957,29.1,12860792,98.2,100950280,770.2
